In [ ]:
# Clone repository 
!git clone https://github.com/IdoXpoz/money-games.git

%cd money-games

In [ ]:
#Switch to another branch if needed
!git fetch
!git checkout almog-work

In [ ]:
# secrets (put the real ones here when running the code)
# If we put them in the code itself, github refuses to push.
HUGGINGFACE_TOKEN = "aaa"
GEMINI_API_KEY = "aaa"

In [ ]:
# Import the experiment runner
from src.experiment.runner import ExperimentRunner

# Initialize and setup
runner = ExperimentRunner(huggingface_token=HUGGINGFACE_TOKEN, gemini_api_key=GEMINI_API_KEY)
runner.setup()


In [ ]:
# Run the full experiment
print("Starting LLM Monetary Priming Experiment...")
print("=" * 50)

# Run experiment (set include_gemini=True to also test Gemini)
results_df = runner.run_open_source_experiment()

print("\nExperiment completed!")
print("=" * 50)


In [ ]:
# Display results
print("Experiment Results Summary:")
print("=" * 50)

# Show a clean summary of results
summary_df = results_df[['model', 'prefix_type', 'paraphrase_index', 'response']].copy()
display(summary_df)

print(f"\nTotal experiments run: {len(results_df)}")
print(f"Models tested: {results_df['model'].nunique()}")
print(f"Prefix conditions: {results_df['prefix_type'].nunique()}")
print(f"Paraphrases tested: {results_df['paraphrase_index'].nunique()}")
print(f"Total combinations per model: {results_df['prefix_type'].nunique()} prefixes × {results_df['paraphrase_index'].nunique()} paraphrases = {results_df['prefix_type'].nunique() * results_df['paraphrase_index'].nunique()}")


In [ ]:
# Analyze decision token probabilities and top tokens (for open source models only)
print("Decision Token Probability Analysis:")
print("=" * 50)

for idx, row in results_df.iterrows():
    if row['decision_tokens'] is not None:
        print(f"\nModel: {row['model']}")
        print(f"Prefix: {row['prefix_type']}")
        print(f"Paraphrase: {row['paraphrase_index']}")
        print(f"Response: {row['response']}")
        print("Decision token probabilities:")
        for token, prob in row['decision_tokens']:
            print(f"  {token}: {prob:.15f}")
        
        # Print top k tokens if available
        if 'top_tokens' in row and row['top_tokens'] is not None:
            print("Top k most probable next tokens:")
            for i, (token, prob) in enumerate(row['top_tokens'], 1):
                print(f"  {i}. '{token}': {prob:.15f}")
        
        print("-" * 30)


In [ ]:
# Save results
runner.save_results("experiment_results.csv")

# Download results file (for Colab)
try:
    from google.colab import files
    files.download("experiment_results.csv")
    print("Results downloaded!")
except ImportError:
    print("Results saved to experiment_results.csv")
